In [57]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
import numpy as np
import time
import datetime
features = pd.read_csv('C:\\Users\\Artem\\Downloads\\features\\features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [78]:
df = features.drop(['duration','tower_status_radiant','barracks_status_radiant','barracks_status_dire','tower_status_dire'], 1)

In [79]:
df.count(axis=0)[df.count(axis=0)<97230]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

In [80]:
df.fillna(value=0,inplace='True')

In [81]:
y=df['radiant_win']

In [82]:
df = df.drop(['start_time','radiant_win'], 1)

In [50]:
from sklearn.cross_validation import KFold
Part=KFold(len(y),n_folds=5,shuffle=True, random_state=1)

In [58]:
start_time = datetime.datetime.now()

clf=GradientBoostingClassifier(n_estimators=30, random_state=241)
scores=cross_val_score(clf,X=df,y=y,scoring="roc_auc",cv=Part)
print 'Time elapsed:', datetime.datetime.now() - start_time
print np.mean(scores)

Time elapsed: 0:01:02.229000
0.689513666842


In [59]:
start_time = datetime.datetime.now()

clf=GradientBoostingClassifier(n_estimators=10, random_state=241)
scores=cross_val_score(clf,X=df,y=y,scoring="roc_auc",cv=Part)
print 'Time elapsed:', datetime.datetime.now() - start_time
print np.mean(scores)

Time elapsed: 0:00:21.640000
0.664832922805


In [60]:
start_time = datetime.datetime.now()

clf=GradientBoostingClassifier(n_estimators=20, random_state=241)
scores=cross_val_score(clf,X=df,y=y,scoring="roc_auc",cv=Part)
print 'Time elapsed:', datetime.datetime.now() - start_time
print np.mean(scores)

Time elapsed: 0:00:40.361000
0.68204033896


In [63]:
from sklearn.preprocessing import StandardScaler
StSc=StandardScaler()
df=StSc.fit_transform(df)

In [66]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(penalty='l2',C=1.0)

In [76]:
for i in range(1,12,2):
    print float(i)/100
    clf=LogisticRegression(penalty='l2',C=float(i)/100)
    scores=cross_val_score(clf,X=df,y=y,scoring="roc_auc",cv=Part)
    print np.mean(scores)

0.01
0.716300040808
0.03
0.716280398534
0.05
0.716276733948
0.07
0.71627509151
0.09
0.716274194984
0.11
0.716273339312


In [83]:
df1=df.drop(['lobby_type','r1_hero', 'r2_hero',  'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero','d3_hero','d4_hero', 'd5_hero'],axis=1)
df1=StSc.fit_transform(df1)

In [85]:
for i in [1,10,100,500]:
    print float(i)/100
    clf=LogisticRegression(penalty='l2',C=float(i)/100)
    scores=cross_val_score(clf,X=df1,y=y,scoring="roc_auc",cv=Part)
    print np.mean(scores)

0.01
0.716329945755
0.1
0.716308344964
1.0
0.716305695435
5.0
0.716305637217


In [104]:
np.max(df['d1_hero'])

112

In [105]:
X_pick = np.zeros((df.shape[0], 112))

for i, match_id in enumerate(df.index):
    for p in xrange(5):
        X_pick[i, df.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [111]:
df2=np.append(df1,X_pick,axis=1)

In [120]:
for i in [6]:
    print float(i)/100
    clf=LogisticRegression(penalty='l2',C=float(i)/100)
    scores=cross_val_score(clf,X=df2,y=y,scoring="roc_auc",cv=Part)
    print np.mean(scores)

0.06
0.751767518326


In [117]:
features_test = pd.read_csv('C:\\Users\\Artem\\Downloads\\features_test\\features_test.csv', index_col='match_id')

In [124]:
test=features_test.drop(['start_time','lobby_type','r1_hero', 'r2_hero',  'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero','d3_hero','d4_hero', 'd5_hero'],axis=1)

In [122]:
X_pick = np.zeros((test.shape[0], 112))

for i, match_id in enumerate(test.index):
    for p in xrange(5):
        X_pick[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [125]:
test.fillna(value=0,inplace='True')
test=StSc.transform(test)
test=np.append(test,X_pick,axis=1)

In [127]:
clf.fit(X=df2,y=y)

LogisticRegression(C=0.06, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [128]:
pred = clf.predict_proba(test)[:, 1]

In [130]:
print np.max(pred),np.min(pred)

0.996497651656 0.00832964728974
